In [4]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [6]:
# read in with pandas
winter1 = pd.read_csv ('daw20-21data.txt')
winter2 = pd.read_csv ('daw21-22data.txt')
winter3 = pd.read_csv ('daw22-23data.txt')

In [8]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [10]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [12]:
precip_days2

,station,datetime,precip(mm),wxcodes
36,DAW,2021-12-02 10:26:00,T,-SN
37,DAW,2021-12-02 10:51:00,T,UP
38,DAW,2021-12-02 10:59:00,0.25,-FZRA
39,DAW,2021-12-02 11:22:00,1.02,FZRA BR
40,DAW,2021-12-02 11:48:00,1.78,-FZRA BR
...,...,...,...,...
2937,DAW,2022-02-25 23:29:00,0.25,-SN
2938,DAW,2022-02-25 23:51:00,0.25,-SN BR
2939,DAW,2022-02-26 00:19:00,T,-SN BR
2940,DAW,2022-02-26 00:26:00,T,-SN


In [14]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_23824\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_23824\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_23824\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [18]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_23824\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_23824\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_23824\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [20]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 51]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 51]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 51]

In [22]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [24]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [28]:
daw = LocationInfo(
    'Rochester', 'New Hampshire', 
    latitude=43.3045,  
    longitude=-70.9756,
    timezone='America/New_Hampshire'
)

daw

LocationInfo(name='Rochester', region='New Hampshire', timezone='America/New_Hampshire', latitude=43.3045, longitude=-70.9756)

In [30]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(daw.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [32]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(daw.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [34]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(daw.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [36]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:57:11  2020-12-01 16:08:54  2020-12-01
1   2020-12-02 06:58:15  2020-12-02 16:08:37  2020-12-02
2   2020-12-03 06:59:18  2020-12-03 16:08:23  2020-12-03
3   2020-12-04 07:00:20  2020-12-04 16:08:11  2020-12-04
4   2020-12-05 07:01:21  2020-12-05 16:08:01  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:28:04  2021-02-24 17:26:38  2021-02-24
86  2021-02-25 06:26:28  2021-02-25 17:27:56  2021-02-25
87  2021-02-26 06:24:51  2021-02-26 17:29:12  2021-02-26
88  2021-02-27 06:23:13  2021-02-27 17:30:29  2021-02-27
89  2021-02-28 06:21:34  2021-02-28 17:31:45  2021-02-28

[90 rows x 3 columns]


In [38]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:56:55  2021-12-01 16:08:58  2021-12-01
1   2021-12-02 06:58:00  2021-12-02 16:08:41  2021-12-02
2   2021-12-03 06:59:03  2021-12-03 16:08:26  2021-12-03
3   2021-12-04 07:00:05  2021-12-04 16:08:13  2021-12-04
4   2021-12-05 07:01:06  2021-12-05 16:08:03  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:28:27  2022-02-24 17:26:20  2022-02-24
86  2022-02-25 06:26:51  2022-02-25 17:27:37  2022-02-25
87  2022-02-26 06:25:14  2022-02-26 17:28:54  2022-02-26
88  2022-02-27 06:23:37  2022-02-27 17:30:10  2022-02-27
89  2022-02-28 06:21:58  2022-02-28 17:31:27  2022-02-28

[90 rows x 3 columns]


In [40]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:56:39  2022-12-01 16:09:03  2022-12-01
1   2022-12-02 06:57:44  2022-12-02 16:08:45  2022-12-02
2   2022-12-03 06:58:48  2022-12-03 16:08:29  2022-12-03
3   2022-12-04 06:59:51  2022-12-04 16:08:16  2022-12-04
4   2022-12-05 07:00:52  2022-12-05 16:08:05  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:28:50  2023-02-24 17:26:01  2023-02-24
86  2023-02-25 06:27:14  2023-02-25 17:27:18  2023-02-25
87  2023-02-26 06:25:38  2023-02-26 17:28:35  2023-02-26
88  2023-02-27 06:24:00  2023-02-27 17:29:52  2023-02-27
89  2023-02-28 06:22:22  2023-02-28 17:31:08  2023-02-28

[90 rows x 3 columns]


In [42]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_23824\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [44]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,DAW,2020-12-01 00:51:00,3.30,RA BR,2020-12-01,2020-12-01 06:57:11,2020-12-01 16:08:54
1,DAW,2020-12-01 01:51:00,2.29,-RA BR,2020-12-01,2020-12-01 06:57:11,2020-12-01 16:08:54
2,DAW,2020-12-01 02:51:00,5.08,+RA BR,2020-12-01,2020-12-01 06:57:11,2020-12-01 16:08:54
3,DAW,2020-12-01 03:51:00,1.27,-RA BR,2020-12-01,2020-12-01 06:57:11,2020-12-01 16:08:54
4,DAW,2020-12-01 04:51:00,1.27,-RA,2020-12-01,2020-12-01 06:57:11,2020-12-01 16:08:54
...,...,...,...,...,...,...,...
227,DAW,2021-02-27 16:51:00,0.51,-SN BR,2021-02-27,2021-02-27 06:23:13,2021-02-27 17:30:29
228,DAW,2021-02-27 17:51:00,1.02,-RA BR,2021-02-27,2021-02-27 06:23:13,2021-02-27 17:30:29
229,DAW,2021-02-27 18:51:00,3.05,-RA BR,2021-02-27,2021-02-27 06:23:13,2021-02-27 17:30:29
230,DAW,2021-02-27 19:51:00,1.27,-RA BR,2021-02-27,2021-02-27 06:23:13,2021-02-27 17:30:29


In [46]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,DAW,2021-12-02 10:51:00,0.0001,UP,2021-12-02,2021-12-02 06:58:00,2021-12-02 16:08:41
1,DAW,2021-12-02 11:51:00,1.7800,FZRA BR,2021-12-02,2021-12-02 06:58:00,2021-12-02 16:08:41
2,DAW,2021-12-02 12:51:00,2.0300,RA BR,2021-12-02,2021-12-02 06:58:00,2021-12-02 16:08:41
3,DAW,2021-12-02 13:51:00,1.2700,-RA BR,2021-12-02,2021-12-02 06:58:00,2021-12-02 16:08:41
4,DAW,2021-12-02 15:51:00,0.0001,-RA FG,2021-12-02,2021-12-02 06:58:00,2021-12-02 16:08:41
...,...,...,...,...,...,...,...
233,DAW,2022-02-25 20:51:00,0.5100,-SN,2022-02-25,2022-02-25 06:26:51,2022-02-25 17:27:37
234,DAW,2022-02-25 21:51:00,0.0001,-SN,2022-02-25,2022-02-25 06:26:51,2022-02-25 17:27:37
235,DAW,2022-02-25 22:51:00,0.2500,-SN,2022-02-25,2022-02-25 06:26:51,2022-02-25 17:27:37
236,DAW,2022-02-25 23:51:00,0.2500,-SN BR,2022-02-25,2022-02-25 06:26:51,2022-02-25 17:27:37


In [48]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,DAW,2022-12-01 00:51:00,7.6200,RA BR,2022-12-01,2022-12-01 06:56:39,2022-12-01 16:09:03
1,DAW,2022-12-01 01:51:00,0.7600,-RA,2022-12-01,2022-12-01 06:56:39,2022-12-01 16:09:03
2,DAW,2022-12-03 15:51:00,0.2500,-RA,2022-12-03,2022-12-03 06:58:48,2022-12-03 16:08:29
3,DAW,2022-12-03 16:51:00,1.0200,-RA BR,2022-12-03,2022-12-03 06:58:48,2022-12-03 16:08:29
4,DAW,2022-12-03 17:51:00,0.7600,-RA BR,2022-12-03,2022-12-03 06:58:48,2022-12-03 16:08:29
...,...,...,...,...,...,...,...
312,DAW,2023-02-26 16:51:00,0.7600,-SN,2023-02-26,2023-02-26 06:25:38,2023-02-26 17:28:35
313,DAW,2023-02-26 17:51:00,0.7600,-SN,2023-02-26,2023-02-26 06:25:38,2023-02-26 17:28:35
314,DAW,2023-02-26 18:51:00,0.2500,-SN,2023-02-26,2023-02-26 06:25:38,2023-02-26 17:28:35
315,DAW,2023-02-26 19:51:00,0.0001,-SN,2023-02-26,2023-02-26 06:25:38,2023-02-26 17:28:35


In [50]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
7,DAW,2020-12-01 07:51:00,1.7800,RA BR,2020-12-01,2020-12-01 06:57:11,2020-12-01 16:08:54
8,DAW,2020-12-01 08:51:00,0.2500,-RA BR,2020-12-01,2020-12-01 06:57:11,2020-12-01 16:08:54
9,DAW,2020-12-05 11:51:00,0.0001,-RA,2020-12-05,2020-12-05 07:01:21,2020-12-05 16:08:01
10,DAW,2020-12-05 12:51:00,1.2700,-RA BR,2020-12-05,2020-12-05 07:01:21,2020-12-05 16:08:01
11,DAW,2020-12-05 13:51:00,2.0300,RA BR,2020-12-05,2020-12-05 07:01:21,2020-12-05 16:08:01
...,...,...,...,...,...,...,...
223,DAW,2021-02-27 12:51:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:23:13,2021-02-27 17:30:29
224,DAW,2021-02-27 13:51:00,1.2700,SN FZFG,2021-02-27,2021-02-27 06:23:13,2021-02-27 17:30:29
225,DAW,2021-02-27 14:51:00,1.5200,SN FZFG,2021-02-27,2021-02-27 06:23:13,2021-02-27 17:30:29
226,DAW,2021-02-27 15:51:00,1.0200,-SN BR,2021-02-27,2021-02-27 06:23:13,2021-02-27 17:30:29


In [52]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_23824\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_23824\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_23824\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [54]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
8,DAW,2020-12-01 08:51:00,0.2500,-RA BR,2020-12-01,2020-12-01 06:57:11,2020-12-01 16:08:54,2020-12-01 07:57:11,2020-12-01 15:08:54
9,DAW,2020-12-05 11:51:00,0.0001,-RA,2020-12-05,2020-12-05 07:01:21,2020-12-05 16:08:01,2020-12-05 08:01:21,2020-12-05 15:08:01
10,DAW,2020-12-05 12:51:00,1.2700,-RA BR,2020-12-05,2020-12-05 07:01:21,2020-12-05 16:08:01,2020-12-05 08:01:21,2020-12-05 15:08:01
11,DAW,2020-12-05 13:51:00,2.0300,RA BR,2020-12-05,2020-12-05 07:01:21,2020-12-05 16:08:01,2020-12-05 08:01:21,2020-12-05 15:08:01
12,DAW,2020-12-05 14:51:00,2.5400,RA BR,2020-12-05,2020-12-05 07:01:21,2020-12-05 16:08:01,2020-12-05 08:01:21,2020-12-05 15:08:01
...,...,...,...,...,...,...,...,...,...
204,DAW,2021-02-19 15:51:00,0.0001,-SN,2021-02-19,2021-02-19 06:35:51,2021-02-19 17:20:08,2021-02-19 07:35:51,2021-02-19 16:20:08
223,DAW,2021-02-27 12:51:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:23:13,2021-02-27 17:30:29,2021-02-27 07:23:13,2021-02-27 16:30:29
224,DAW,2021-02-27 13:51:00,1.2700,SN FZFG,2021-02-27,2021-02-27 06:23:13,2021-02-27 17:30:29,2021-02-27 07:23:13,2021-02-27 16:30:29
225,DAW,2021-02-27 14:51:00,1.5200,SN FZFG,2021-02-27,2021-02-27 06:23:13,2021-02-27 17:30:29,2021-02-27 07:23:13,2021-02-27 16:30:29


In [56]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [58]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [60]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.8725774647887327
2021-2022: 0.7583202020202019
2022-2023: 1.2128654320987653


In [62]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,DAW,2020-12-01 00:51:00,3.30,RA BR,2020-12-01,2020-12-01 06:57:11,2020-12-01 16:08:54
1,DAW,2020-12-01 01:51:00,2.29,-RA BR,2020-12-01,2020-12-01 06:57:11,2020-12-01 16:08:54
2,DAW,2020-12-01 02:51:00,5.08,+RA BR,2020-12-01,2020-12-01 06:57:11,2020-12-01 16:08:54
3,DAW,2020-12-01 03:51:00,1.27,-RA BR,2020-12-01,2020-12-01 06:57:11,2020-12-01 16:08:54
4,DAW,2020-12-01 04:51:00,1.27,-RA,2020-12-01,2020-12-01 06:57:11,2020-12-01 16:08:54
...,...,...,...,...,...,...,...
222,DAW,2021-02-23 01:51:00,0.51,-RA BR,2021-02-23,2021-02-23 06:29:39,2021-02-23 17:25:21
228,DAW,2021-02-27 17:51:00,1.02,-RA BR,2021-02-27,2021-02-27 06:23:13,2021-02-27 17:30:29
229,DAW,2021-02-27 18:51:00,3.05,-RA BR,2021-02-27,2021-02-27 06:23:13,2021-02-27 17:30:29
230,DAW,2021-02-27 19:51:00,1.27,-RA BR,2021-02-27,2021-02-27 06:23:13,2021-02-27 17:30:29


In [64]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_23824\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_23824\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_23824\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [66]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [68]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [70]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.7964099999999998
2021-2022: 0.43848653846153857
2022-2023: 0.602969411764706
